Testbed for bots, by Arseny, based on Henry's testing_grounds.

In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

import draftsimtools as ds

In [2]:
# Load M19 drafts
raw_drafts = ds.load_drafts("../../data/m19_2.csv")

# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

In [3]:
with open('../../data/Allsets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
    
jsonSubset = mtgJSON['M19']['cards']

thisSet = {card['name'] : card for card in jsonSubset}
nameList = pd.DataFrame(thisSet.keys(),columns = ['Name']) # Instead of relying on cvs, get names from json

In [4]:
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, so needs this 1st arg

In [5]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.
Processing draft: 10000.
Processing draft: 20000.
Processing draft: 30000.
Processing draft: 40000.
Processing draft: 50000.
Processing draft: 60000.
Processing draft: 70000.
Processing draft: 80000.
Processing draft: 90000.
Processing draft: 100000.


In [6]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

### Now this part below will just create weird bots and test them

But for now I can as well sketch a random bot here.

In [ ]:
# Creates a bot that implements current Draftsim logic
# draftsim_bot = ds.DraftsimBot(rating_dict)

In [18]:
nameList[nameList['Name']=='Aether Tunnel'].index[0]
# Knight_of_the_Tusk

4

In [67]:
nameList['newName'] = nameList.Name.str.replace(' ','')
nameList.newName = nameList.newName.str.lower()
nCardsInSet = len(nameList)
nCardsInSet

300

In [85]:
nameList.newName[nameList.newName.str[0]=='t']

258             takevengeance
259          talonsofwildwood
260             tatteredmummy
261              tectonicrift
262     tezzeret'sgatebreaker
263         tezzeret'sstrider
264    tezzeretartificemaster
265    tezzeretcruelmachinist
266           thornlieutenant
267           thornhidewolves
268                      thud
269               timbergorge
270             titanicgrowth
271           tolarianscholar
272           tormentingvoice
273               totallylost
274           tranquilexpanse
275       transmogrifyingwand
276              trumpetblast
277           trustypackbeast
278          two-headedzombie
Name: Name, dtype: object

In [87]:
nameList[nameList.newName=='totallylost'].index[0]

273

In [96]:
for iDraft in range(11,12): #range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    counter = 0
    for pack in draft:
        counter += 1        
        print(counter)
        x = np.zeros(nCardsInSet+1)
        for card in pack:
            cardName = card.replace('_','').lower()
            try:
                pos = nameList[nameList.newName==cardName].index[0]
            except:
                pos = nCardsInSet
            x[pos] += 1
            print(cardName,pos)
        print(pack)
        print(x)

1
lathliss 300
departeddeckhand 55
terrorofqalsisma 300
exclusionmage 81
rabidbite 196
mysticarchaeologist 169
risefromthegrave 209
pegasuscourser 185
onewiththemachine 180
inspiredcharge 127
plummet 191
onakkeogre 179
onakkeogre 179
macabrewaltz 151
totallylost 273
['Lathliss', 'Departed_Deckhand', '_Terror_of_Qal_Sisma', 'Exclusion_Mage', 'Rabid_Bite', 'Mystic_Archaeologist', 'Rise_from_the_Grave', 'Pegasus_Courser', 'One_with_the_Machine', 'Inspired_Charge', 'Plummet', 'Onakke_Ogre', 'Onakke_Ogre', 'Macabre_Waltz', 'Totally_Lost']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 

inspiredcharge 127
mindrot 163
disperse 61
['Divination', 'Star-Crowned_Stag', 'Reassembling_Skeleton', 'Cavalry_Drillmaster', 'Fire_Elemental', 'Inspired_Charge', 'Mind_Rot', 'Disperse']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.

In [ ]:
# Gets bot accuracy for some packs
tester = ds.BotTester(test)
tester.evaluate_bots([draftsim_bot, sgd_bot], ["Draftsim2018", "Draftsim2018SGD"])
tester.write_evaluations()